In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Embedding, ELU, Add, Softmax, LayerNormalization
from tensorflow.keras.models import Model
import numpy as np
import tensorflow.keras as keras
import importlib
import dataprocessor
from tqdm import tqdm
import os


In [18]:
import models
importlib.reload(models)
importlib.reload(dataprocessor)
from models import *
from dataprocessor import *

target_vol = 21, 5e-2
macd_timescales = {(8, 24), (16, 28), (32, 96)}
rtn_timescales = {1, 21, 63, 126, 252}
timesteps = 5

folder_path = 'data'
files = [f[:-5] for f in os.listdir(folder_path) if f.endswith('.xlsx')]
files = [file for file in files if file not in ("CC00.NYB", "LB00.CME", "ES00.CME", "NQ00.CME", "YM00.CBT")]
data_list = []

for file in tqdm(files, desc="处理文件中。。"):
    data = pd.read_excel(f"data/{file}.xlsx")[["日期", "收盘价(元)"]]
    data = data.rename(columns={"日期": "date", "收盘价(元)": "close"}).sort_values("date")
    data["side_info"] = file
    try:
        data = generate_features(data, macd_timescales, rtn_timescales)
    except:
        print(file)
    data_list.append(data)

print("Process data into a dataset")
targetset = generate_tf_dataset(data_list, timesteps)

处理文件中。。: 100%|██████████| 12/12 [00:51<00:00,  4.33s/it]


Process data into a dataset


In [ ]:
def encode_context_info(dataset):
    features, _ = next(iter(dataset))
    shape = features.shape
    context_info_values= []
    for _, context_info in dataset:
        context_info_values.append(context_info['side_info'].numpy().decode('utf-8'))

    # Step 2: One-hot encode the context_info categories
    unique_context_info = list(set(context_info_values))
    context_info_map = {val: idx for idx, val in enumerate(unique_context_info)}
    num_classes = len(unique_context_info)

    def one_hot_encode(value):
        one_hot = np.zeros(num_classes)
        one_hot[context_info_map[value]] = 1
        return one_hot

    encoded_data = []
    print("编码中...")
    for _, context_info in dataset:
        side_info = context_info['side_info'].numpy().decode('utf-8')
        one_hot_encoded = one_hot_encode(side_info)
        encoded_data.append(one_hot_encoded)

    encoded_data = tf.ragged.constant(encoded_data).to_tensor(default_value=0.0)
    expanded_tensor = tf.expand_dims(encoded_data, axis=1)
    encoded_data = tf.tile(expanded_tensor, [1, shape[0], 1])
    return encoded_data

encode_context_info(targetset)

In [ ]:
for features, labels in targetset.take(10):
    print("Features:", features)
    print("Labels:", labels)

In [5]:
batch_size = 5
time_steps = 2  # 每个样本的时间步
sequence_length = 4

encoding_size = 17  # 17 种合约
hidden_dim = 8

x = tf.random.uniform((batch_size, time_steps, sequence_length))
s_indices = tf.tile(tf.range(batch_size)[:, tf.newaxis], [1, time_steps])
s = tf.one_hot(s_indices + 6, depth=encoding_size)

model = BaselineNeuralForecaster(sequence_length, hidden_dim, encoding_size)
model.build((x.shape, s.shape))
result = model(x, s)